#Cross Validating Regression


##After this Lecture you Should Be Able To:
Discuss the following topics and know how to implement them in sklearn
1. [Cross Validation](#cross-validation)
1. [Regression with sklearn](#regression-with-sklearn)
1. [Fitting a polynomial](#fitting-a-polynomial)

You should also be familiar with the standard graphs for these issues and be able to explain them and understand how they affect your own work.
1. [Bias and Variance](#bias-and-variance)
1. [Overfitting and Underfitting](#overfitting-and-underfitting)


## Cross Validation

Before you begin fitting a model on a new dataset you should, almost always, split your initial dataset into a "train" dataset and a "test" dataset. You will use the train dataset to build your model and the test dataset to measure the model's success.

You should generally keep 10-50% of the data for the test set and use the rest for training. 

You should always split your data as randomly as possible. It should be obvious to you that the slightest inclusion of a nonrandom process in the selection of the training set can skew model parameters. Data is frequently sorted in some way (by date or even by the value you are trying to predict).

*Never* just split your data into the first 90% and the remaining 10%. Lucky for us, there is a nice method implemented in scipy that splits the dataset randomly for us called[test_train_split](http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.train_test_split.html). If you do not happen to have this tool in the language you're working in (i.e. Java), you'll need to make it.


### KFold Cross Validation

One thing about doing standard cross validation is that, as discussed above, your score can depend on the random split. We can get a more accurate value of the error by using *KFold Cross Validation*. 

Basically, we break the data into k groups. We take one of these groups and make it the test set. We then train the data on the remaining groups and calculate the error on the test group. We can repeat this process k times and average all the results. This gives us a more accurate picture of the error.

## Regression with sklearn

There are several good modules with implementations of regression. We've
used
[statsmodels](http://statsmodels.sourceforge.net/devel/generated/statsmodels.regression.linear_model.OLS.html).
Today we will be using [sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html).
[numpy](docs.­scipy.­org/­doc/­numpy/­reference/­generated/­numpy.­polyfit.­html) and
[scipy](http://docs.scipy.org/doc/scipy-0.13.0/reference/generated/scipy.stats.linregress.html)
also have implmentations.

Resources:
* [sklearn documentation](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)
* [sklearn example](http://scikit-learn.org/0.11/auto_examples/linear_model/plot_ols.html)

For all `sklearn` modules, the `fit` method is used to train and the `score`
method is used to test. You can also use the `predict` method to see the
predicted y values.

#### Example

This is the general workflow for working with sklearn. Any algorithm we use from sklearn will have the same form.

```python
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split

# Load data from csv file
df = pd.read_csv('data/housing_prices.csv')
X = df[['square_feet', 'num_rooms']].values
y = df['price'].values

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

# Run Linear Regression
regr = LinearRegression()
regr.fit(X_train, y_train)
print "Intercept:", regr.intercept_
print "Coefficients:", regr.coef_
print "R^2 error:", regr.score(X_test, y_test)
predicted_y = regr.predict(X_test)
```

 `LinearRegression` is a class and you have to create an instance of it. If there are any parameters to the model, you should set them when you instantiate the object. For example, with `LinearRegression`, you can choose whether to normalize you data:

 ```python
 regr = LinearRegression(normalize=True)    # the default is False
 ```

You should call the `fit` method once. Here you give it the training data and it will train your model. Once you have that, you can get the coefficients for the equation (`intercept_` and `coef_`) and also get the score for your test set (`score` method). You can also get the predicted values for any new data you would like to give to the model (`predict` method).

Here's an example using kfold:

```python
from sklearn import cross_validation
kf = cross_validation.KFold(X.shape[0], n_folds=5, shuffle=True)
results = []
for train_index, test_index in kf:
    regr = LinearRegression()
    regr.fit(X[train_index], y[train_index])
    results.append(regr.score(X[test_index], y[test_index]))
print "average score:", np.mean(results)
```


## Fitting a polynomial

Oftentimes you'll notice that you're data isn't linear and that really you should be fitting a higher degree polynomial to the line. This is called *underfitting*, which we'll get to later.

![quadratic](images/quadratic.png)

So how do we do this? We can use the same algorithm, we just need to modify our features. Let's look at the one feature world for a minute. We have data that looks something like this:

|     x |     y |
| ----- | ----- |
|     3 |     8 |
|     4 |    17 |
|     7 |    40 |
|     9 |    78 |
|    11 |   109 |

 For linear regression, we are trying to find the `b` and `c` that minimize the error in the following equation:

    bx + c = y

To do a *quadratic* regression instead of a *linear* regression, we instead want to find the optimal `a`, `b` and `c` in this equation:

    ax^2 + bx + c = y

We can just add a new feature to our feature matrix by computing `x^2`:

|     x |   x^2 |    y |
| ----- | ----- | ----- |
|     3 |     9 |     8 |
|     4 |    16 |    17 |
|     7 |    49 |    40 |
|     9 |    81 |    78 |
|    11 |   121 |   109 |

Now you can do linear regression with these features. If there's a linear relationship between `x^2` and `y` that means there's a quadratic relationship between `x` and `y`.

If you have more than one feature, you need to do all the combinations. If you start with two features, `x` and `z`, to do the order 2 polynomial, you will need to add these features: `x^2`, `z^2`, `xz`.

In `sklearn`, you should use `PolynomialFeatures` for generating these additional features ([documentation](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html#sklearn.preprocessing.PolynomialFeatures)). Here's how you would modify the above example to include polynomial features up to degree 3:

```python
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split

# Load data is identical as above
df = pd.read_csv('data/housing_prices.csv')
X = df[['square_feet', 'num_rooms']]
y = df['price']

# Add the polynomial features
poly = PolynomialFeatures(3)
X_new = poly.fit_transform(X)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.15)

# Run Linear Regression (the same as above)
regr = LinearRegression()
regr.fit(X_train, y_train)
print "Intercept:", regr.intercept_
print "Coefficients:", regr.coef_
print "R^2 error:", regr.score(X_test, y_test)
predicted_y = regr.predict(X_test)
```

Here's sklearn's [example](http://scikit-learn.org/stable/auto_examples/linear_model/plot_polynomial_interpolation.html). They use a `pipeline` to make the code a little simpler (with the same functionality).

In `numpy`, you can use the `polyfit` function ([documentation](http://docs.scipy.org/doc/numpy/reference/generated/numpy.polyfit.html)).

The discussion below will give insight into how to determine which degree of polynomial to pick.


## Bias and Variance

We first want to discuss two key terms: *bias* and *variance*.

#### Error due to bias
Imagine that you construct your model with the same process several hundred times. A *biased* model would center around the incorrect solution. How you collect data can lead to bias (for example, you only get user data from San Francisco and try to use your model for your whole userbase).

#### Error due to variance
Again, imagine that you construct your model several hundred times. A model with high *variance* would have wildly different results each time. The main contributor to high variance is insufficient data or that what you're trying to predict isn't actually correlated to your features.

To see the bais variance tradeoff visually:

![bias variance](images/bias_variance.png)

Note that both high bias or high variance are bad. Note that high variance is worse than it sounds since you will only be constructing the model once, so with high variance there's a low probability that your model will be near the optimal one.


## Overfitting and underfitting

Let's get back to fitting the polynomial. Take a look at the following example with three potential curves fit to the data.

![underfitting overfitting](images/underfitting_overfitting_1.png)

In the first graph, we've fit a line to the data. It clearly doesn't fully represent the data. This is called *underfitting*. This represents high *bias* since the error will be consistently high. It also represents low *variance* since with new data, we would still construct approximately the same model.

In the third graph, we've fit a polynomial of high degree to the data. Notice how it accurately gets every point but we'd say this does not accurately represent the data. This is called *overfitting*. This represents high *variance*. If we got new data, we would construct a wildly different model. This also represents low *bias* since the error is very low.

The one in the middle is the optimal choice.

You can see this graphically:

![bias variance](images/bias_variance_graph.png)

Model complexity in our case is the degree of the polynomial.

Another way of viewing this is by comparing the error on the training set with the error on the test set. When you fit a model, it minimizes the error on the training set. An overfit model can reduce this to 0. However, what we really care about it how well it does on a new test set, since we want our model to perform well on unseen data. This paradigm is represented with the following graph.

![overfitting underfitting](images/graph_overfitting_underfitting.png)

You can see on the left side of the graph that the data is *underfit*: both the train error and the test error are high. On the right hand side of the graph, the data is *overfit*: the train error is low but the test error is high. The best model is where the train error is lowest, around a degree 5 polynomial.


## Regularization: How to deal with overfitting

Having lots of features can lead to overfitting. When we make it into a polynomial, we are adding *a lot* of features. *Regularization* is a way of penalizing coefficients for being large.

## L1 regularization [Lasso regression](http://statweb.stanford.edu/~tibs/lasso/simple.html)
Math description: Regularization adds a random variable (for L1, a Laplacian) to the hat (estimate) matrix so that it can be inverted.

  1. Turns most regressors to zeros
  2. Uses a Laplacian prior

### When to use: 

  1. Large sparse data sets, many regressors will become zero. 
  2. When you have many regressors but are unsure of the important ones.

<u>Pros</u>:

  1. Good for recovering sparse datasets
  2. Reduce overfitting

<u>Cons</u>:

  1. More difficult to interpret
  2. Loss of predictive power
  3. Large estimation error for non-sparse data.

## L2 regularization (Ridge regression):
1. Ridge regression suppresses the influence of the leading regressors lightly and the lagging regressors  heavily. 
2. Uses a Gaussian prior

### When to use: 
  1. When you have many regressors but are unsure of the important ones
  2. Non-sparse data. 

<u>Pros</u>:
  1. Good for recovering non-sparse signals. 
  2. Reduce overfitting.
  3. Less variance than the OLS estimator [reference](http://tamino.wordpress.com/2011/02/12/ridge-regression/)

<u>Cons</u>:

  1. The new estimates of the regressors are lower than the OLS estimates [reference](http://tamino.wordpress.com/2011/02/12/ridge-regression/)
  2. Loss of predictive power

## <u>More references</u>:

* [Ridge regression](http://tamino.wordpress.com/2011/02/12/ridge-regression/)
* [Lasso regression](http://statweb.stanford.edu/~tibs/lasso/simple.html)
* [Difference between L1 and L2](http://www.quora.com/Machine-Learning/What-is-the-difference-between-L1-and-L2-regularization), Aleks Jakulins answer. 
* [Matrix for of regression models](http://global.oup.com/booksites/content/0199268010/samplesec3)
* [The statistics bible](http://statweb.stanford.edu/~tibs/ElemStatLearn/printings/ESLII_print10.pdf), chapter 3
* [stats.stackexchange: Ridge vs. LASSO](http://stats.stackexchange.com/questions/866/when-should-i-use-lasso-vs-ridge)
* [MetaOptimize: L1 vs. L2](http://metaoptimize.com/qa/questions/5205/when-to-use-l1-regularization-and-when-l2)

## Classification

We will cover these concepts in more depth, but there are some basics to understand around how to evaluate a classifier.

### Measuring success

So how do we measure how well our model does? Just like with regression, we need to split the data in a training set and a test set and measure our success based on how well it does on the test set.

#### Accuracy
The simplest measure is **accuracy**. This is the number of correct predictions over the total number of predictions. It's the percent you predicted correctly. In `sklearn`, this is what the `score` method calculates.

#### Shortcomings of Accuracy
Accuracy is often a good first glance measure, but it has many shortcomings. If the classes are unbalanced, accuracy will not measure how well you did at predicting. Say you are trying to predict whether or not an email is spam. Only 2% of emails are in fact spam emails. You could get 98% accuracy by always predicting not spam. This is a great accuracy but a horrible model!

#### Confusion Matrix
We can get a better picture our model but looking at the confusion matrix. We get the following four metrics:

* **True Positives (TP)**: Correct positive predictions
* **False Positives (FP)**: Incorrect positive predictions (false alarm)
* **True Negatives (TN)**: Correct negative predictions
* **False Negatives (FN)**: Incorrect negative predictions (a miss)

|            | Predicted Yes  | Predicted No   |
| ---------- | -------------- | -------------- |
| Actual Yes | True positive  | False negative |
| Actual No  | False positive | True negative  |


#### Precision, Recall and F1
Instead of accuracy, there are some other scores we can calculate:

* **Precision**: A measure of how good your positive predictions are
    ```
    Precison = TP / (TP + FP)
             = TP / (predicted yes)
    ```
* **Recall**: A measure of how well you predict positive cases. Aka *sensitivity*.
    ```
    Recall = TP / (TP + FN) 
           = TP / (actual yes)
    ```
* **F1 Score**: The harmonic mean of Precision and Recall
    ```
    F1 = 2 / (1/Precision + 1/Recall)
       = 2 * Precision * Recall / (Precision + Recall)
       = 2TP / (2TP + FN + FP)
    ```

Accuracy can also be written in this notation:
    ```
    Accuracy = (TP + TN) / (TP + FP + TN + FN)
    ```
